In [1]:
import numpy
import numpy as np
import pandas
import pandas as pd
from numba import njit
import sys,glob
import zipfile, os, time
import urllib3
from datetime import datetime, timedelta
import glob
import h5py
import plotly.express as px
sys.path.append(os.getcwd()+'/../cds-backend/code/')
sys.path.append(os.getcwd()+'/../harvest/code/')
from harvest_convert_to_netCDF_newfixes import write_dict_h5
import cds_eua3 as eua
eua.logging_set_level(30)
import xarray as xr

import cdsapi, zipfile, os, time
#import schedule
import copy
from shutil import copyfile
import multiprocessing
sys.path.append(os.getcwd()+'/rasotools-master/')
import rasotools

    34	humidity	atmospheric	surface; upper-air	dew point depression	K	Dew point depression is also called dew point deficit. It is the amount by which the air temperature exceeds its dew point temperature. Dew point temperature is the temperature at which a parcel of air reaches saturation upon being cooled at constant pressure and specific humidity.

    36	humidity	atmospheric	surface; upper-air	dew point temperature	K	Dew point temperature is the temperature at which a parcel of air reaches saturation upon being cooled at constant pressure and specific humidity.

    37	humidity	atmospheric	surface; upper-air	ice bulb temperature	K	TBD

    38	humidity	atmospheric	surface; upper-air	relative humidity	percent	TBD

    39	humidity	atmospheric	surface; upper-air	specific humidity	g kg-1	specific means per unit mass. Specific humidity is the mass fraction of water vapor in (moist) air.



    104	wind	atmospheric	surface; upper-air	eastward wind speed	m s-1	Eastward indicates a vector component which is positive when directed eastward (negative westward). Wind is defined as a two-dimensional (horizontal) air velocity vector,  with no vertical component. (Vertical motion in the atmosphere has the standard name upward air velocity.)

    105	wind	atmospheric	surface; upper-air	northward wind speed	m s-1	Northward indicates a vector component which is positive when directed northward (negative southward). Wind is defined as a two-dimensional (horizontal) air velocity vector,  with no vertical component. (Vertical motion in the atmosphere has the standard name upward air velocity.)

    106	wind	atmospheric	surface; upper-air	wind from direction	degree	direction from which the wind is blowing Lot 1 uses dd  - WMO abbrev.

    107	wind	atmospheric	surface; upper-air	wind speed	m s-1	Speed is the magnitude of velocity. Wind is defined as a two-dimensional (horizontal) air velocity vector,  with no vertical component. (Vertical motion in the atmosphere has the standard name upward air velocity.) The wind speed is the magnitude of the wind velocity. Lot 1 uses ff  - WMO abbrev.

    137	humidity	atmospheric	surface; upper air	air dewpoint	K	Dewpoint measurement (from profile measurement)

    138	humidity	atmospheric	surface; upper air	relative humidity	1	Relative humidity (from profile measurement)

    139	wind	atmospheric	surface; upper air	eastward wind speed	m s-1	Eastward wind speed (from profile measurement)

    140	wind	atmospheric	surface; upper air	northward wind speed	m s-1	Northward wind speed (from profile measurement)

In [2]:
files = glob.glob('/raid60/scratch/uli/resorted_files_correct/*.nc')

In [3]:
def check_for_missing(file, checkvars: list = [85, 38, 106, 107], convert: bool = False):
    missing = []
    with eua.CDMDataset(file) as data:
        for i in checkvars:
            if not str(i) in data.recordindices.keys():
                missing.append(i)
        if not convert:
            return missing
        else: 
            convert_missing(file, missing)

In [41]:
#
# todo: 
# variablen nur einmal lesen, und nicht für jedern convertierweg nochmal
# für alle methoden sammeln und dann das, was mehrfach da ist, aussortieren
# das übrige dann mit dem vergleichen, was schon im Datensatz steht -> auffüllen 
# fertig
#



def convert_missing(file, missing, destination: str = './converted'):
    with eua.CDMDataset(file) as data:
        print('loading data')
        df = data.to_dataframe(groups='observations_table', variables=['observed_variable', 'observation_value', 'date_time', 'z_coordinate', 'report_id', 'latitude', 'longitude'])
        print('loading complete')
        
# --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#

    if 106 in missing or 107 in missing: # wd and/or ws is missing -> calculate it from u/v
        u = df[df.observed_variable == 104]
        v = df[df.observed_variable == 105]
        
        if(len(u) != len(v)):
            print('not of equal length')
            cleaned_u = u[0:0]
            cleaned_v = v[0:0]
            for i in v.report_id.drop_duplicates():
                inter_u = u[u.report_id == i]
                inter_v = v[v.report_id == i]
                cleaned_u = cleaned_u.append(inter_u[inter_u.z_coordinate.isin(inter_v.z_coordinate)])
                cleaned_v = cleaned_v.append(inter_v[inter_v.z_coordinate.isin(inter_u.z_coordinate)]) 
            print('equal length - start processing')
            u = np.asarray(cleaned_u.observation_value)
            v = np.asarray(cleaned_v.observation_value)
        else:
            print('equal length - start processing')
            u = np.asarray(u.observation_value)
            v = np.asarray(v.observation_value)

        ws = np.sqrt(u ** 2 + v ** 2)
        wd = 90 - np.arctan2(-v, -u) * 180 / np.pi - 180.
        wd = np.where(wd > 0., wd, 360.+wd)
        return wd, ws
        #add
        #set conversion flag
    if 104 in missing or 105 in missing: # u and/or v is missing -> calculate it from wd and ws
        wd = df[df.observed_variable == 106]
        ws = df[df.observed_variable == 107]
        
        if(len(wd) != len(ws)):
            print('not of equal length')
            cleaned_wd = wd[0:0]
            cleaned_ws = ws[0:0]
            for i in ws.report_id.drop_duplicates():
                inter_wd = wd[wd.report_id == i]
                inter_ws = ws[ws.report_id == i]
                cleaned_wd = cleaned_wd.append(inter_wd[inter_wd.z_coordinate.isin(inter_ws.z_coordinate)])
                cleaned_ws = cleaned_ws.append(inter_ws[inter_ws.z_coordinate.isin(inter_wd.z_coordinate)])
            print('equal length - start processing')
            wd = np.asarray(cleaned_wd.observation_value)
            ws = np.asarray(cleaned_ws.observation_value)
        else:
            print('equal length - start processing')
            wd = np.asarray(wd.observation_value)
            ws = np.asarray(ws.observation_value)

        u = ws * np.cos(np.radians(wd))
        v = ws * np.sin(np.radians(wd)) 
        return u, v
        #add
        #set conversion flag
            
# --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#

    if 38 in missing: # rh is missing -> calculate it from 
        hums = [34, 36, 37, 39]
        missing_hum_vars = check_for_missing(file, checkvars = hums)
        for i in missing_hum_vars: hums.remove(i)
        print(hums)
        rhall = []
        if 34 in hums: # dew point depression
            print('with 34')
            t = df[df.observed_variable == 85]
            hum = df[df.observed_variable == 34]
            
            cleaned_t = t[0:0]
            cleaned_hum = hum[0:0]
            for i in hum.report_id.drop_duplicates():
                inter_t = t[t.report_id == i]
                inter_hum = hum[hum.report_id == i]
                cleaned_t = cleaned_t.append(inter_t[inter_t.z_coordinate.isin(inter_hum.z_coordinate)])
                cleaned_hum = cleaned_hum.append(inter_hum[inter_hum.z_coordinate.isin(inter_t.z_coordinate)])
        
            t = cleaned_t.observation_value.to_xarray()
            dpd = cleaned_hum.observation_value.to_xarray()

            rh = rasotools.met.convert.to_rh(t, dpd=dpd)
            rhall.append(rh)
            #add
            #set conversion flag
        if 36 in hums: # dew point
            print('with 36')
            t = df[df.observed_variable == 85]
            hum = df[df.observed_variable == 36]

            cleaned_t = t[0:0]
            cleaned_hum = hum[0:0]
            for i in hum.report_id.drop_duplicates():
                inter_t = t[t.report_id == i]
                inter_hum = hum[hum.report_id == i]
                cleaned_t = cleaned_t.append(inter_t[inter_t.z_coordinate.isin(inter_hum.z_coordinate)])
                cleaned_hum = cleaned_hum.append(inter_hum[inter_hum.z_coordinate.isin(inter_t.z_coordinate)])

            dpd = cleaned_t.observation_value.subtract(np.asarray(cleaned_hum.observation_value))
            t = cleaned_t.observation_value.to_xarray()
            dpd = dpd.to_xarray()

            rh = rasotools.met.convert.to_rh(t,dpd=dpd)
            rhall.append(rh)
            #add
            #set conversion flag
        if 39 in hums: # specific humidity
            print('with 39')
            t = df[df.observed_variable == 85]
            hum = df[df.observed_variable == 39]

            cleaned_t = t[0:0]
            cleaned_hum = hum[0:0]
            for i in hum.report_id.drop_duplicates():
                inter_t = t[t.report_id == i]
                inter_hum = hum[hum.report_id == i]
                cleaned_t = cleaned_t.append(inter_t[inter_t.z_coordinate.isin(inter_hum.z_coordinate)])
                cleaned_hum = cleaned_hum.append(inter_hum[inter_hum.z_coordinate.isin(inter_t.z_coordinate)])

            p = cleaned_t.z_coordinate.to_xarray()
            t = cleaned_t.observation_value.to_xarray()
            sh = cleaned_hum.observation_value.to_xarray()

            rh = rasotools.met.convert.to_rh(temp=t, spec_humi=sh, press=p)
            rhall.append(rh)
            #add
            #set conversion flag
        else:
            print('-')
            print('NO HUMIDITY VARIABLE AVAILABLE - COULD NOT CONVERT')
            print('-')
        return(rhall)
            
# --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#

    if 34 in missing: # dew point depression is missing -> calculate it from
        hums = [38, 36, 37, 39]
        missing_hum_vars = check_for_missing(file, checkvars = hums)
        for i in missing_hum_vars: hums.remove(i)
        print(hums)
        dpdall = []
        if 38 in hums and not 85 in missing: # relative humidity
            print('with 38')
            t = df[df.observed_variable == 85]
            hum = df[df.observed_variable == 38]

            cleaned_t = t[0:0]
            cleaned_hum = hum[0:0]
            for i in hum.report_id.drop_duplicates():
                inter_t = t[t.report_id == i]
                inter_hum = hum[hum.report_id == i]
                cleaned_t = cleaned_t.append(inter_t[inter_t.z_coordinate.isin(inter_hum.z_coordinate)])
                cleaned_hum = cleaned_hum.append(inter_hum[inter_hum.z_coordinate.isin(inter_t.z_coordinate)])

            t = cleaned_t.observation_value.to_xarray()
            p = cleaned_t.z_coordinate.to_xarray()
            rh = cleaned_hum.observation_value.to_xarray()

            dpd = rasotools.met.convert.to_dpd(temp=t,press=p,rel_humi=rh)
            dpdall.append(dpd)
            #add
            #set conversion flag
        if 36 in hums: # dew point
            print('with 36')
            t = df[df.observed_variable == 85]
            hum = df[df.observed_variable == 36]

            cleaned_t = t[0:0]
            cleaned_hum = hum[0:0]
            for i in hum.report_id.drop_duplicates():
                inter_t = t[t.report_id == i]
                inter_hum = hum[hum.report_id == i]
                cleaned_t = cleaned_t.append(inter_t[inter_t.z_coordinate.isin(inter_hum.z_coordinate)])
                cleaned_hum = cleaned_hum.append(inter_hum[inter_hum.z_coordinate.isin(inter_t.z_coordinate)])


            dpd = cleaned_t.observation_value.subtract(np.asarray(cleaned_hum.observation_value))
            dpd = dpd.to_xarray()
            dpdall.append(dpd)
            #add
            #set conversion flag
        if 39 in hums: # specific humidity
            print('with 39')
            t = df[df.observed_variable == 85]
            hum = df[df.observed_variable == 39]

            cleaned_t = t[0:0]
            cleaned_hum = hum[0:0]
            for i in hum.report_id.drop_duplicates():
                inter_t = t[t.report_id == i]
                inter_hum = hum[hum.report_id == i]
                cleaned_t = cleaned_t.append(inter_t[inter_t.z_coordinate.isin(inter_hum.z_coordinate)])
                cleaned_hum = cleaned_hum.append(inter_hum[inter_hum.z_coordinate.isin(inter_t.z_coordinate)])
                
            p = cleaned_t.z_coordinate.to_xarray()
            t = cleaned_t.observation_value.to_xarray()
            sh = cleaned_hum.observation_value.to_xarray()
            print(p.values)

            dpd = rasotools.met.convert.to_dpd(spec_humi=sh,press=p,temp=t)
            dpdall.append(dpd)
            #add
            #set conversion flag
        else:
            print('-')
            print('NO HUMIDITY VARIABLE AVAILABLE - COULD NOT CONVERT')
            print('-')
        return dpdall
            
# --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#

    if 36 in missing: # dew point is missing -> calculate it from
        hums = [34, 38, 37, 39]
        missing_hum_vars = check_for_missing(file, checkvars = hums)
        for i in missing_hum_vars: hums.remove(i)
        print(hums)
        dpall = []
        if 38 in hums and not 85 in missing: # relative humidity
            print('with 38')
            t = df[df.observed_variable == 85]
            hum = df[df.observed_variable == 38]

            cleaned_t = t[0:0]
            cleaned_hum = hum[0:0]
            for i in hum.report_id.drop_duplicates():
                inter_t = t[t.report_id == i]
                inter_hum = hum[hum.report_id == i]
                cleaned_t = cleaned_t.append(inter_t[inter_t.z_coordinate.isin(inter_hum.z_coordinate)])
                cleaned_hum = cleaned_hum.append(inter_hum[inter_hum.z_coordinate.isin(inter_t.z_coordinate)])

            t = cleaned_t.observation_value.to_xarray()
            p = cleaned_t.z_coordinate.to_xarray()
            rh = cleaned_hum.observation_value.to_xarray()

            dp = rasotools.met.convert.to_dewpoint(temp=t,press=p,rel_humi=rh)
            dpall.append(dp)
            #add
            #set conversion flag
        if 34 in hums: # dew point depression
            print('with 34')
            t = df[df.observed_variable == 85]
            hum = df[df.observed_variable == 34]

            cleaned_t = t[0:0]
            cleaned_hum = hum[0:0]
            for i in hum.report_id.drop_duplicates():
                inter_t = t[t.report_id == i]
                inter_hum = hum[hum.report_id == i]
                cleaned_t = cleaned_t.append(inter_t[inter_t.z_coordinate.isin(inter_hum.z_coordinate)])
                cleaned_hum = cleaned_hum.append(inter_hum[inter_hum.z_coordinate.isin(inter_t.z_coordinate)])

            dp = cleaned_t.observation_value.subtract(np.asarray(cleaned_hum.observation_value))
            dp = dp.to_xarray()
            dpall.append(dp)
            #add
            #set conversion flag
        if 39 in hums: # specific humidity
            print('with 39')
            t = df[df.observed_variable == 85]
            hum = df[df.observed_variable == 39]

            cleaned_t = t[0:0]
            cleaned_hum = hum[0:0]
            for i in hum.report_id.drop_duplicates():
                inter_t = t[t.report_id == i]
                inter_hum = hum[hum.report_id == i]
                cleaned_t = cleaned_t.append(inter_t[inter_t.z_coordinate.isin(inter_hum.z_coordinate)])
                cleaned_hum = cleaned_hum.append(inter_hum[inter_hum.z_coordinate.isin(inter_t.z_coordinate)])

            p = cleaned_t.z_coordinate.to_xarray()
            sh = cleaned_hum.observation_value.to_xarray()
            dp = rasotools.met.convert.to_dewpoint(spec_humi=sh,press=p)
            dpall.append(dp)
            #add
            #set conversion flag
        else:
            print('-')
            print('NO HUMIDITY VARIABLE AVAILABLE - COULD NOT CONVERT')
            print('-')
        return dpall
            
# --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#

    if 39 in missing: # specific humidity is missing -> calculate it from
        hums = [34, 38, 37, 36]
        missing_hum_vars = check_for_missing(file, checkvars = hums)
        for i in missing_hum_vars: hums.remove(i)
        print(hums)
        shall = []
        if 38 in hums and not 85 in missing: # relative humidity
            print('with 38')
            t = df[df.observed_variable == 85]
            hum = df[df.observed_variable == 38]

            cleaned_t = t[0:0]
            cleaned_hum = hum[0:0]
            for i in hum.report_id.drop_duplicates():
                inter_t = t[t.report_id == i]
                inter_hum = hum[hum.report_id == i]
                cleaned_t = cleaned_t.append(inter_t[inter_t.z_coordinate.isin(inter_hum.z_coordinate)])
                cleaned_hum = cleaned_hum.append(inter_hum[inter_hum.z_coordinate.isin(inter_t.z_coordinate)])

            t = cleaned_t.observation_value.to_xarray()
            p = cleaned_t.z_coordinate.to_xarray()
            rh = cleaned_hum.observation_value.to_xarray()

            sh = rasotools.met.convert.to_sh(temp=t, press=p, rel_humi=rh)
            shall.append(sh)
            #add
            #set conversion flag
        if 34 in hums: # dew point depression
            print('with 34')
            t = df[df.observed_variable == 85]
            hum = df[df.observed_variable == 34]

            cleaned_t = t[0:0]
            cleaned_hum = hum[0:0]
            for i in hum.report_id.drop_duplicates():
                inter_t = t[t.report_id == i]
                inter_hum = hum[hum.report_id == i]
                cleaned_t = cleaned_t.append(inter_t[inter_t.z_coordinate.isin(inter_hum.z_coordinate)])
                cleaned_hum = cleaned_hum.append(inter_hum[inter_hum.z_coordinate.isin(inter_t.z_coordinate)])

            t = cleaned_t.observation_value.to_xarray()
            p = cleaned_t.z_coordinate.to_xarray()
            dpd = cleaned_hum.observation_value.to_xarray()

            sh = rasotools.met.convert.to_sh(dpd=dpd, press=p, temp=t)
            shall.append(sh)
            #add
            #set conversion flag
        if 36 in hums: # dew point
            print('with 36')
            t = df[df.observed_variable == 85]
            hum = df[df.observed_variable == 36]

            cleaned_t = t[0:0]
            cleaned_hum = hum[0:0]
            for i in hum.report_id.drop_duplicates():
                inter_t = t[t.report_id == i]
                inter_hum = hum[hum.report_id == i]
                cleaned_t = cleaned_t.append(inter_t[inter_t.z_coordinate.isin(inter_hum.z_coordinate)])
                cleaned_hum = cleaned_hum.append(inter_hum[inter_hum.z_coordinate.isin(inter_t.z_coordinate)])

            dpd = cleaned_t.observation_value.subtract(np.asarray(cleaned_hum.observation_value))

            t = cleaned_t.observation_value.to_xarray()
            p = cleaned_t.z_coordinate.to_xarray()
            dpd = dpd.to_xarray()
            sh = rasotools.met.convert.to_sh(dpd=dpd, press=p, temp=t)
            shall.append(sh)
            #add
            #set conversion flag
        else:
            print('-')
            print('NO HUMIDITY VARIABLE AVAILABLE - COULD NOT CONVERT')
            print('-')
        return shall

In [42]:
t0 = time.time()
convert_missing('/raid60/scratch/uli/resorted_files_correct/0-20000-0-11035_CEUAS_merged_v0.nc', missing=[106])
print("Time elapsed: ", time.time()-t0, "s")

loading data
loading complete
equal length - start processing
Time elapsed:  41.01807165145874 s


In [33]:
with eua.CDMDataset('/raid60/scratch/uli/resorted_files_correct/0-20000-0-11019_CEUAS_merged_v0.nc') as data:
    df = data.to_dataframe(groups='observations_table', variables=['observed_variable', 'observation_value', 'date_time', 'z_coordinate', 'report_id', 'latitude', 'longitude'])
    print(data.recordindices['85'][-1] - data.recordindices['85'][0])
    print(data.recordindices['38'][-1] - data.recordindices['38'][0])
    print(data.recordindices['104'][-1] - data.recordindices['104'][0])
    print(data.recordindices['105'][-1] - data.recordindices['105'][0])
    
t = df[df.observed_variable == 85]
hum = df[df.observed_variable == 38]

cleaned_t = t[0:0]
cleaned_hum = hum[0:0]
for i in hum.report_id.drop_duplicates():
    inter_t = t[t.report_id == i]
    inter_hum = hum[hum.report_id == i]
    cleaned_t = cleaned_t.append(inter_t[inter_t.z_coordinate.isin(inter_hum.z_coordinate)])
    cleaned_hum = cleaned_hum.append(inter_hum[inter_hum.z_coordinate.isin(inter_t.z_coordinate)])

t = cleaned_t.observation_value.to_xarray()
p = cleaned_t.z_coordinate.to_xarray()
rh = cleaned_hum.observation_value.to_xarray()

rasotools.met.convert.to_sh(temp=t, press=p, rel_humi=rh)

<xarray.DataArray 'sh' (index: 5122)>
array([0.000783, 0.00078 , 0.000755, ..., 0.004638, 0.004835, 0.004583],
      dtype=float32)
Coordinates:
  * index    (index) int64 15680 15681 15682 15683 ... 20813 20814 20815 20816
Attributes:
    origin:              t,rh,p
    esat:                HylandWexler
    standard_name:       specific_humidity
    long_name:           specific humidity
    units:               kg/kg
    enhancement_factor:  yes
    precision:           6

In [ ]:
with eua.CDMDataset('/raid60/scratch/uli/resorted_files_correct/0-20000-0-11035_CEUAS_merged_v0.nc') as data:
    df = data.to_dataframe(groups='observations_table', variables=['observed_variable', 'observation_value', 'date_time', 'z_coordinate', 'report_id', 'latitude', 'longitude'])
wind = df[df.observed_variable == 106]
wind = wind.append(df[df.observed_variable == 107])
errorrep = []
errorlev = []
for i in wind.report_id:
    dat1 = wind[wind.report_id == i] 
    for j in dat1.z_coordinate:
        dat2 = dat1[dat1.z_coordinate == j]
        wdir = df[df.observed_variable == 106]
        wspe = df[df.observed_variable == 107]
        if len(wdir) != len(wspe):
            errorrep.append(i)
            errorlev.append(j)


In [21]:
with eua.CDMDataset('/raid60/scratch/uli/resorted_files_correct/0-20000-0-11019_CEUAS_merged_v0.nc') as data:
    ri_t = data.recordindices['85'][:]
#     ri_34 = data.recordindices['34'][:]
    ri_36 = data.recordindices['36'][:]
    ri_38 = data.recordindices['38'][:]
    ri_39 = data.recordindices['39'][:]
    obsvar = data.observations_table.observed_variable[:]
    obsval = data.observations_table.observation_value[:]
    pres = data.observations_table.z_coordinate[:]
    repid = data.observations_table.report_id[:]
    
t = obsval[ri_t[0]:ri_t[-1]-1]
trep = repid[ri_t[0]:ri_t[-1]-1]
p_t = pres[ri_t[0]:ri_t[-1]-1]
hum39 = obsval[ri_39[0]:ri_39[-1]-1]
hum39rep = repid[ri_39[0]:ri_39[-1]-1]
p_hum39 = pres[ri_39[0]:ri_39[-1]-1]

In [78]:
# from numba import jit
# @jit(nopython=True) 
def get_to_same_len(a, b, repa, repb, pa, pb):
#     anew = a[np.in1d(repa, np.unique(repb))]
#     panew = pa[np.in1d(repa, np.unique(repb))]
#     bnew = b[np.in1d(repb, np.unique(repa))]
#     pbnew = pb[np.in1d(repb, np.unique(repa))]
    bout = np.empty(0)
    pbout = np.empty(0)
    for i in np.unique(repa[np.in1d(repa, np.unique(repb))]):
        bout = np.concatenate((bout, bnew[np.in1d(repb, i)]))
        pbout = np.concatenate((pbout, pbnew[np.in1d(repb, i)]))
    aout = np.empty(0)
    paout = np.empty(0)
    for i in np.unique(repb[np.in1d(repb, np.unique(repa))]):
        aout = np.concatenate((aout, anew[np.in1d(repa, i)]))
        paout = np.concatenate((paout, panew[np.in1d(repa, i)]))
    return aout, bout, paout, pbout

In [83]:
trep_dc = np.empty(len(trep))
for i in range(len(trep)):
    trep_dc[i] = (int(b"".join(trep[i]).decode()))
hum39rep_dc = np.empty(len(hum39rep))
for i in range(len(hum39rep)):
    hum39rep_dc[i] = (int(b"".join(hum39rep[i]).decode()))

In [84]:
a, b, pa, pb = get_to_same_len(t, hum39, trep_dc, hum39rep_dc, p_t, p_hum39)

In [85]:
print(len(a))
print(len(b))
print(len(pa))
print(len(pb))

0
5168
0
5168


In [177]:
tneu = t[np.in1d(trep_dc, np.unique(hum39rep_dc))]
p_tneu = p_t[np.in1d(trep_dc, np.unique(hum39rep_dc))]
hum39neu = hum39[np.in1d(hum39rep_dc, np.unique(trep_dc))]
p_hum39neu = p_hum39[np.in1d(hum39rep_dc, np.unique(trep_dc))]
hum39out = []
p_hum39out = []
for i in np.unique(trep_dc[np.in1d(trep_dc, np.unique(hum39rep_dc))]):
    hum39out = hum39out + (list(hum39neu[np.in1d(hum39rep_dc, i)]))
    p_hum39out = p_hum39out + (list(p_hum39neu[np.in1d(hum39rep_dc, i)]))
tout = []
p_tout = []
for i in np.unique(hum39rep_dc[np.in1d(hum39rep_dc, np.unique(trep_dc))]):
    tout = tout + (list(tneu[np.in1d(trep_dc, i)]))
    p_tout = p_tout + (list(p_tneu[np.in1d(trep_dc, i)]))


In [179]:
len(p_tout)

5136

In [180]:
len(p_hum39out)

5433

In [114]:
len(t[mask])

3321027

In [115]:
len(hum39)

2819680

In [ ]:
test = np.array([0, 1, 2, 5, 0])

states = [0, 2]

mask = np.in1d(test, states)

mask
array([ True, False,  True, False,  True])

test[mask]
array([0, 2, 0])

In [117]:
trep_dc = []
for i in trep:
    trep_dc.append(b"".join(i).decode())
hum39rep_dc = []
for i in hum39rep:
    hum39rep_dc.append(b"".join(i).decode())

In [101]:
len(np.unique(trep_dc))

50552

In [102]:
hum39rep_dc = []
for i in hum39rep:
    hum39rep_dc.append(b"".join(i).decode())

In [103]:
len(np.unique(hum39rep_dc))

38442

In [108]:
np.where(np.asarray(hum39rep_dc).isin(np.unique(trep_dc)))

AttributeError: 'numpy.ndarray' object has no attribute 'isin'

In [54]:
rh

<xarray.DataArray 'rh' (index: 5122)>
array([0.152205, 0.151341, 0.141188, ..., 0.572029, 0.481809, 0.432807],
      dtype=float32)
Coordinates:
  * index    (index) int64 15680 15681 15682 15683 ... 20813 20814 20815 20816
Attributes:
    units:               1
    standard_name:       relativ_humidity
    long_name:           relative humidity
    esat:                HylandWexler
    origin:              t,q,p
    enhancement_factor:  yes
    precision:           6

In [52]:
sh

,observed_variable,observation_value,date_time,z_coordinate,report_id,latitude,longitude
10246,39,0.000780,1972-10-30 14:00:00,70000.0,20000000000,45.529999,-122.68
10247,39,0.000777,1972-10-30 14:00:00,70900.0,20000000000,45.529999,-122.68
10248,39,0.000752,1972-10-30 14:00:00,74600.0,20000000000,45.529999,-122.68
10249,39,0.000810,1972-10-30 14:00:00,75000.0,20000000000,45.529999,-122.68
10250,39,0.000819,1972-10-30 14:00:00,79600.0,20000000000,45.529999,-122.68
...,...,...,...,...,...,...,...
15674,39,0.004183,1973-06-18 19:00:00,89400.0,20000000311,45.529999,-122.68
15675,39,0.004216,1973-06-18 19:00:00,90000.0,20000000311,45.529999,-122.68
15676,39,0.004615,1973-06-18 19:00:00,95000.0,20000000311,45.529999,-122.68
15677,39,0.004809,1973-06-18 19:00:00,100000.0,20000000311,45.529999,-122.68


In [ ]:
sh[]

In [ ]:
# #
# # todo: 
# # variablen nur einmal lesen, und nicht für jedern convertierweg nochmal
# # für alle methoden sammeln und dann das, was mehrfach da ist, aussortieren
# # das übrige dann mit dem vergleichen, was schon im Datensatz steht -> auffüllen 
# # fertig
# #



# def convert_missing(file, missing, destination: str = './converted'):
#     with eua.CDMDataset(file) as data:
#         df = data.to_dataframe(groups='observations_table', variables=['observed_variable', 'observation_value', 'date_time', 'z_coordinate', 'report_id', 'latitude', 'longitude'])
        
# # --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#

#     if 106 in missing or 107 in missing: # wd and/or ws is missing -> calculate it from u/v
#         u = np.asarray(df[df.observed_variable == 104].observation_value)
#         v = np.asarray(df[df.observed_variable == 105].observation_value)
#         ws = np.sqrt(u ** 2 + v ** 2)
#         wd = 90 - np.arctan2(-v, -u) * 180 / np.pi - 180.
#         wd = np.where(wd > 0., wd, 360.+wd)
#         return wd, ws
#         #add
#         #set conversion flag
#     if 104 in missing or 105 in missing: # u and/or v is missing -> calculate it from wd and ws
#         wd = np.asarray(df[df.observed_variable == 106].observation_value)
#         ws = np.asarray(df[df.observed_variable == 107].observation_value)
#         u = ws * np.cos(np.radians(wd))
#         v = ws * np.sin(np.radians(wd)) 
#         return u, v
#         #add
#         #set conversion flag
            
# # --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#

#     #
#     # convert from every source and merge the rh outputs? maximize the searched variable -> removed elif and made if everywher
#     #
#     if 38 in missing: # rh is missing -> calculate it from 
#         hums = [34, 36, 37, 39]
#         missing_hum_vars = check_for_missing(file, checkvars = hums)
#         for i in missing_hum_vars: hums.remove(i)
#         print(hums)
# #         if 34 in hums: # dew point depression
# #             print('with 34')
# #             t = df[df.observed_variable == 85]
# #             hum = df[df.observed_variable == 34]
            
# #             cleaned_t = t[0:0]
# #             cleaned_hum = hum[0:0]
# #             for i in hum.report_id.drop_duplicates():
# #                 inter_t = t[t.report_id == i]
# #                 inter_hum = hum[hum.report_id == i]
# #                 cleaned_t = cleaned_t.append(inter_t[inter_t.z_coordinate.isin(inter_hum.z_coordinate)])
# #                 cleaned_hum = cleaned_hum.append(inter_hum[inter_hum.z_coordinate.isin(inter_t.z_coordinate)])
        
# #             t = cleaned_t.observation_value.to_xarray()
# #             dpd = cleaned_hum.observation_value.to_xarray()

# #             rh = rasotools.met.convert.to_rh(t, dpd=dpd)
# #             return rh
# #             #add
# #             #set conversion flag
#         if 36 in hums: # dew point
#             print('with 36')
#             t = df[df.observed_variable == 85]
#             hum = df[df.observed_variable == 36]

#             cleaned_t = t[0:0]
#             cleaned_hum = hum[0:0]
#             for i in hum.report_id.drop_duplicates():
#                 inter_t = t[t.report_id == i]
#                 inter_hum = hum[hum.report_id == i]
#                 cleaned_t = cleaned_t.append(inter_t[inter_t.z_coordinate.isin(inter_hum.z_coordinate)])
#                 cleaned_hum = cleaned_hum.append(inter_hum[inter_hum.z_coordinate.isin(inter_t.z_coordinate)])

#             dpd = cleaned_t.observation_value.subtract(np.asarray(cleaned_hum.observation_value))
#             t = cleaned_t.observation_value.to_xarray()
#             dpd = dpd.to_xarray()

#             rh = rasotools.met.convert.to_rh(t,dpd=dpd)
#             return rh
#             #add
#             #set conversion flag
#         if 39 in hums: # specific humidity
#             print('with 39')
#             t = df[df.observed_variable == 85]
#             hum = df[df.observed_variable == 39]

#             cleaned_t = t[0:0]
#             cleaned_hum = hum[0:0]
#             for i in hum.report_id.drop_duplicates():
#                 inter_t = t[t.report_id == i]
#                 inter_hum = hum[hum.report_id == i]
#                 cleaned_t = cleaned_t.append(inter_t[inter_t.z_coordinate.isin(inter_hum.z_coordinate)])
#                 cleaned_hum = cleaned_hum.append(inter_hum[inter_hum.z_coordinate.isin(inter_t.z_coordinate)])

#             p = cleaned_t.z_coordinate.to_xarray()
#             t = cleaned_t.observation_value.to_xarray()
#             sh = cleaned_hum.observation_value.to_xarray()

#             rh = rasotools.met.convert.to_rh(temp=t, spec_humi=sh, press=p)
#             return rh
#             #add
#             #set conversion flag
#         else:
#             print('-')
#             print('NO HUMIDITY VARIABLE AVAILABLE - COULD NOT CONVERT')
#             print('-')
            
# # --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#

#     #
#     # convert from every source and merge the rh outputs? maximize the searched variable -> removed elif and made if everywher
#     #
#     if 34 in missing: # dew point depression is missing -> calculate it from
#         hums = [38, 36, 37, 39]
#         missing_hum_vars = check_for_missing(file, checkvars = hums)
#         for i in missing_hum_vars: hums.remove(i)
            
#         if 38 in hums and not 85 in missing: # relative humidity
#             t = df[df.observed_variable == 85]
#             rh = df[df.observed_variable == 38]

#             cleaned = t[0:0]
#             for i in rh.report_id.drop_duplicates():
#                 inter = t[t.report_id == i]
#                 for j in rh[rh.report_id == i].z_coordinate:
#                     cleaned = cleaned.append(inter[inter.z_coordinate == j])

#             t = cleaned.observation_value.to_xarray()
#             p = cleaned.z_coordinate.to_xarray()
#             rh = rh.observation_value.to_xarray()

#             dpd = rasotools.met.convert.to_dpd(temp=t,press=p,rel_humi=rh)
#             return dpd
#             #add
#             #set conversion flag
#         if 36 in hums: # dew point
#             t = df[df.observed_variable == 85]
#             dp = df[df.observed_variable == 36]

#             cleaned = t[0:0]
#             for i in dp.report_id.drop_duplicates():
#                 inter = t[t.report_id == i]
#                 for j in dp[dp.report_id == i].z_coordinate:
#                     cleaned = cleaned.append(inter[inter.z_coordinate == j])

#             dpd = cleaned.observation_value.subtract(np.asarray(dp.observation_value))
#             dpd = dpd.to_xarray()
#             return dpd
#             #add
#             #set conversion flag
#         if 39 in hums: # specific humidity
#             t = df[df.observed_variable == 85]
#             sh = df[df.observed_variable == 39]

#             cleaned = t[0:0]
#             for i in sh.report_id.drop_duplicates():
#                 inter = t[t.report_id == i]
#                 for j in sh[sh.report_id == i].z_coordinate:
#                     cleaned = cleaned.append(inter[inter.z_coordinate == j])

#             p = cleaned.z_coordinate.to_xarray()
#             t = cleaned.observation_value.to_xarray()
#             sh = sh.observation_value.to_xarray()

#             dpd = rasotools.met.convert.to_dpd(spec_humi=sh,press=p,temp=t)
#             return dpd
#             #add
#             #set conversion flag
#         else:
#             print('-')
#             print('NO HUMIDITY VARIABLE AVAILABLE - COULD NOT CONVERT')
#             print('-')
            
# # --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#

#     #
#     # convert from every source and merge the rh outputs? maximize the searched variable -> removed elif and made if everywher
#     #
#     if 36 in missing: # dew point is missing -> calculate it from
#         hums = [34, 38, 37, 39]
#         missing_hum_vars = check_for_missing(file, checkvars = hums)
#         for i in missing_hum_vars: hums.remove(i)
            
#         if 38 in hums and not 85 in missing: # relative humidity
#             t = df[df.observed_variable == 85]
#             rh = df[df.observed_variable == 38]

#             cleaned = t[0:0]
#             for i in rh.report_id.drop_duplicates():
#                 inter = t[t.report_id == i]
#                 for j in rh[rh.report_id == i].z_coordinate:
#                     cleaned = cleaned.append(inter[inter.z_coordinate == j])

#             t = cleaned.observation_value.to_xarray()
#             p = cleaned.z_coordinate.to_xarray()
#             rh = rh.observation_value.to_xarray()

#             dp = rasotools.met.convert.to_dewpoint(temp=t,press=p,rel_humi=rh)
#             return dp
#             #add
#             #set conversion flag
#         if 34 in hums: # dew point depression
#             t = df[df.observed_variable == 85]
#             dpd = df[df.observed_variable == 34]

#             cleaned = t[0:0]
#             for i in dpd.report_id.drop_duplicates():
#                 inter = t[t.report_id == i]
#                 cleaned.append(inter)
# #                 for j in dpd[dpd.report_id == i].z_coordinate:
# #                     cleaned = cleaned.append(inter[inter.z_coordinate == j])

#             dp = cleaned.observation_value.subtract(np.asarray(dpd.observation_value))
#             dp = dp.to_xarray()
#             return dp
#             #add
#             #set conversion flag
#         if 39 in hums: # specific humidity
#             t = df[df.observed_variable == 85]
#             sh = df[df.observed_variable == 39]

#             cleaned = t[0:0]
#             for i in sh.report_id.drop_duplicates():
#                 inter = t[t.report_id == i]
#                 for j in sh[sh.report_id == i].z_coordinate:
#                     cleaned = cleaned.append(inter[inter.z_coordinate == j])

#             p = cleaned.z_coordinate.to_xarray()
#             sh = sh.observation_value.to_xarray()
#             dp = rasotools.met.convert.to_dewpoint(spec_humi=sh,press=p)
#             return dp
#             #add
#             #set conversion flag
#         else:
#             print('-')
#             print('NO HUMIDITY VARIABLE AVAILABLE - COULD NOT CONVERT')
#             print('-')
            
# # --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#

#     #
#     # convert from every source and merge the rh outputs? maximize the searched variable -> removed elif and made if everywher
#     #
#     if 39 in missing: # specific humidity is missing -> calculate it from
#         hums = [34, 38, 37, 36]
#         missing_hum_vars = check_for_missing(file, checkvars = hums)
#         for i in missing_hum_vars: hums.remove(i)
            
#         if 38 in hums and not 85 in missing: # relative humidity
#             t = df[df.observed_variable == 85]
#             rh = df[df.observed_variable == 38]

#             cleaned = t[0:0]
#             for i in rh.report_id.drop_duplicates():
#                 inter = t[t.report_id == i]
#                 for j in rh[rh.report_id == i].z_coordinate:
#                     cleaned = cleaned.append(inter[inter.z_coordinate == j])

#             t = cleaned.observation_value.to_xarray()
#             p = cleaned.z_coordinate.to_xarray()
#             rh = rh.observation_value.to_xarray()

#             sh = rasotools.met.convert.to_sh(temp=t, press=p, rel_humi=rh)
#             return sh
#             #add
#             #set conversion flag
#         if 34 in hums: # dew point depression
#             t = df[df.observed_variable == 85]
#             dpd = df[df.observed_variable == 34]

#             cleaned = t[0:0]
#             for i in dpd.report_id.drop_duplicates():
#                 inter = t[t.report_id == i]
#                 for j in dpd[dpd.report_id == i].z_coordinate:
#                     cleaned = cleaned.append(inter[inter.z_coordinate == j])

#             t = cleaned.observation_value.to_xarray()
#             p = cleaned.z_coordinate.to_xarray()
#             dpd = dpd.observation_value.to_xarray()

#             sh = rasotools.met.convert.to_sh(dpd=dpd, press=p, temp=t)
#             return sh
#             #add
#             #set conversion flag
#         if 36 in hums: # dew point
#             t = df[df.observed_variable == 85]
#             dp = df[df.observed_variable == 36]

#             cleaned = t[0:0]
#             for i in dp.report_id.drop_duplicates():
#                 inter = t[t.report_id == i]
#                 for j in dp[dp.report_id == i].z_coordinate:
#                     cleaned = cleaned.append(inter[inter.z_coordinate == j])

#             dpd = cleaned.observation_value.subtract(np.asarray(dp.observation_value))

#             t = cleaned.observation_value.to_xarray()
#             p = cleaned.z_coordinate.to_xarray()
#             dpd = dpd.to_xarray()
#             sh = rasotools.met.convert.to_sh(dpd=dpd, press=p, temp=t)
#             return sh
#             #add
#             #set conversion flag
#         else:
#             print('-')
#             print('NO HUMIDITY VARIABLE AVAILABLE - COULD NOT CONVERT')
#             print('-')